In [1]:
import itertools
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim_models 
import bertopic

from gensim import corpora
from gensim.models import CoherenceModel,LdaModel,LsiModel
from sklearn.metrics import silhouette_samples, silhouette_score

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

c:\Users\Daito\miniconda3\envs\NLP\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Daito\miniconda3\envs\NLP\Lib\site-packages\catalogue\__init__.py:123: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):


## Chargement des données 

In [2]:
def force_format(texts):
    return [str(t) for t in texts]

def compute_word_occurences(texts):
    words = itertools.chain.from_iterable(texts)
    word_count = pd.Series(words).value_counts()
    word_count = pd.DataFrame({"Word": word_count.index, "Count": word_count.values})
    return word_count

def get_l_texts(text_file): #text_file is a .txt file from preprocessing to avoid doing it again
    l_texts=[]
    with open(text_file, "r") as f:
        line = f.readlines()
        list_line = [l.strip() for l in line]
        for l in list_line:
            l_texts.append(ast.literal_eval(l))
    return l_texts

In [3]:
dataset = pd.read_json("News_Category_Dataset_v2.json", lines=True, dtype={"headline": str})
texts = force_format(dataset["headline"])
l_texts = get_l_texts("l_texts.txt")
print(l_texts[:10],"\n",texts[:10],"\n",dataset["headline"][:10])

[['mass_shooting', 'texas', 'week', 'tv'], ['smith', 'join', 'diplo', 'nicky', 'jam', 'world_cup', 'official', 'song'], ['hugh', 'grant', 'marries', 'time', 'age'], ['jim_carrey', 'blasts', 'castrato', 'adam', 'schiff', 'democrats', 'artwork'], ['julianna', 'margulie', 'donald', 'poop', 'bag', 'pick', 'dog'], ['morgan_freeman', 'devastate', 'sexual_harassment', 'claim', 'undermine', 'legacy'], ['donald', 'lovin', 'mcdonald', 'jingle', 'tonight', 'bit'], ['watch', 'amazon', 'prime', 'week'], ['mike', 'myers', 'reveal', 'fourth', 'austin', 'power', 'film'], ['watch', 'hulu', 'week']] 
 ['There Were 2 Mass Shootings In Texas Last Week, But Only 1 On TV', "Will Smith Joins Diplo And Nicky Jam For The 2018 World Cup's Official Song", 'Hugh Grant Marries For The First Time At Age 57', "Jim Carrey Blasts 'Castrato' Adam Schiff And Democrats In New Artwork", 'Julianna Margulies Uses Donald Trump Poop Bags To Pick Up After Her Dog', "Morgan Freeman 'Devastated' That Sexual Harassment Claims Cou

In [4]:
# Create a dictionary
dictionary = corpora.Dictionary(l_texts)
# Create a corpus
corpus = [dictionary.doc2bow(text) for text in l_texts]

print("Number of unique tokens: {}".format(len(dictionary)))
print("Number of documents: {}".format(len(corpus)))

Number of unique tokens: 47649
Number of documents: 200853


## Pour les modèles disponibles dans la librairie gensim

### LDA

In [5]:
result={"num_topics":[], "coherence_score_cv":[],"coherence_score_npmi":[]}

for n in range(50, 65):
    lda_model = LdaModel(corpus, num_topics=n, id2word=dictionary)
    coherence_model_cv= CoherenceModel(model=lda_model, texts=l_texts, dictionary=dictionary, coherence='c_v')
    coherence_score_cv = coherence_model_cv.get_coherence()
    coherence_model_npmi= CoherenceModel(model=lda_model, texts=l_texts, dictionary=dictionary, coherence='c_npmi')
    coherence_score_npmi = coherence_model_npmi.get_coherence()
    print("Number of topics: ", n, " Coherence Score: ", coherence_score_npmi,coherence_score_cv)
    result["num_topics"].append(n)
    result["coherence_score_cv"].append(coherence_score_cv)
    result["coherence_score_npmi"].append(coherence_score_npmi)

Number of topics:  50  Coherence Score:  -0.2824209588809716 0.5098044333816348
Number of topics:  51  Coherence Score:  -0.28828241735395443 0.5181689460125675
Number of topics:  52  Coherence Score:  -0.2874787327215478 0.5153331178821003
Number of topics:  53  Coherence Score:  -0.2943429399351826 0.5255018014061624
Number of topics:  54  Coherence Score:  -0.29591946141787323 0.5268862888920597
Number of topics:  55  Coherence Score:  -0.2960636395784447 0.5284503792971005
Number of topics:  56  Coherence Score:  -0.3000395592161475 0.5323435274476381
Number of topics:  57  Coherence Score:  -0.29919743955948835 0.5319902171309276
Number of topics:  58  Coherence Score:  -0.2971918680648036 0.5283476888252082
Number of topics:  59  Coherence Score:  -0.29820176195152787 0.5307283765564802
Number of topics:  60  Coherence Score:  -0.30102524437381867 0.5353576336774502
Number of topics:  61  Coherence Score:  -0.2973235104383298 0.5288722685081988
Number of topics:  62  Coherence Sc

In [6]:
optimal_n=result["num_topics"][np.argmax(result["coherence_score_cv"])]
lda_model = LdaModel(corpus, num_topics=n, id2word=dictionary)
coherence_model_cv= CoherenceModel(model=lda_model, texts=l_texts, dictionary=dictionary, coherence='c_v')
coherence_score_cv = coherence_model_cv.get_coherence()
coherence_model_npmi= CoherenceModel(model=lda_model, texts=l_texts, dictionary=dictionary, coherence='c_npmi')
coherence_score_npmi = coherence_model_npmi.get_coherence()
print("Optimal number of topics: ", optimal_n, " Coherence Score: ", coherence_score_npmi,coherence_score_cv)

Optimal number of topics:  60  Coherence Score:  -0.2963964682921284 0.5238451755756386


In [7]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
40     0.273228 -0.393936       1        1  3.120898
7      0.121276  0.060614       2        1  2.526671
36     0.114935  0.054518       3        1  2.422017
51     0.184520  0.204718       4        1  2.264360
12     0.086480  0.032894       5        1  2.118721
...         ...       ...     ...      ...       ...
2     -0.064822 -0.012017      60        1  1.162177
26    -0.089962 -0.014965      61        1  1.104601
21    -0.097759 -0.015726      62        1  1.090094
57    -0.035732 -0.007576      63        1  1.069262
0     -0.075777 -0.013380      64        1  1.026531

[64 rows x 5 columns], topic_info=           Term          Freq         Total Category  logprob  loglift
204       photo  27454.000000  27454.000000  Default  30.0000  30.0000
305       video  14917.000000  14917.000000  Default  29.0000  29.0000
294        make   8629.000000   8629.000000  Default  28.0000  28.0000
496         day   7678.000000   7678.000000  Default  27.0000  27.0000
1550    divorce   5602.000000   5602.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
768       cameo     70.262442     72.252034  Topic64  -5.0411   4.5511
4499      loser     61.461771     63.451358  Topic64  -5.1749   4.5471
4500        nyt     53.305953     55.295540  Topic64  -5.3172   4.5423
1192  boy_scout     48.253104     50.242692  Topic64  -5.4168   4.5386
444    interest     42.194139     44.183726  Topic64  -5.5510   4.5329

[1952 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1144     28  0.988144    aaron
1163     38  0.995406  abandon
3195     56  0.983082      abc
4347     13  0.983172  ability
601      54  0.990211   abroad
...     ...       ...      ...
623      52  0.998602     york
1063     15  0.998599    young
1236     29  0.990522    youth
1122     32  0.984424   zombie
2767      4  0.973939      zoo

[1919 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[41, 8, 37, 52, 13, 44, 16, 64, 62, 18, 15, 17, 7, 26, 6, 54, 56, 39, 2, 55, 61, 14, 50, 59, 57, 42, 60, 63, 34, 24, 4, 43, 5, 35, 38, 20, 49, 28, 23, 31, 10, 9, 11, 40, 19, 12, 30, 53, 51, 45, 33, 47, 29, 48, 25, 36, 46, 32, 21, 3, 27, 22, 58, 1])

### LSI


In [8]:
lsi_model = LsiModel(corpus, num_topics=optimal_n, id2word=dictionary)
coherence_model_cv= CoherenceModel(model=lsi_model, texts=l_texts, dictionary=dictionary, coherence='c_v')
coherence_score_cv = coherence_model_cv.get_coherence()
coherence_model_npmi= CoherenceModel(model=lsi_model, texts=l_texts, dictionary=dictionary, coherence='c_npmi')
coherence_score_npmi = coherence_model_npmi.get_coherence()
print("Number of topics: ", optimal_n, " Coherence Score: ", coherence_score_npmi,coherence_score_cv)

c:\Users\Daito\miniconda3\envs\NLP\Lib\site-packages\gensim\models\lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.
  sparsetools.csc_matvecs(
c:\Users\Daito\miniconda3\envs\NLP\Lib\site-packages\scipy\linalg\_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.
  out = (1 - tri(m.shape[0], m.shape[1], k - 1, m.dtype.char)) * m
c:\Users\Daito\miniconda3\envs\NLP\Lib\site-packages\scipy\linalg\_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprecated as of SciPy 1.11.0 and will be removed in v1.13.0. Please use numpy.(tri/tril/triu) instead.
  out = (1 - tri(m.shape[0], m.shape[1], k - 1, m.dtype.char)) * m
c:\Users\Daito\miniconda3\envs\NLP\Lib\site-packages\scipy\linalg\_special_matrices.py:154: DeprecationWarning: 'tri'/'tril/'triu' are deprec

Number of topics:  60  Coherence Score:  -0.0426014760788819 0.23731591431451662


In [ ]:
# Visualize the topics
#Not possible with LSI

## BERTOPIC

In [ ]:
representation=bertopic.representation.KeyBERTInspired()
model_trained_representation= bertopic.BERTopic(representation_model=representation,nr_topics=optimal_n)
topics,probs = model_trained_representation.fit_transform(dataset['headline'])

In [ ]:
dictionary = corpora.Dictionary(l_texts)
coherence_model= CoherenceModel(model=model_trained_representation, texts=l_texts, dictionary=dictionary, coherence='c_v')
coherence_score_cv = coherence_model.get_coherence()
coherence_model= CoherenceModel(model=model_trained_representation, texts=l_texts, dictionary=dictionary, coherence='c_npmi')
coherence_score_c_npmi = coherence_model.get_coherence()
print("Coherence Score: ", coherence_score_cv,coherence_score_c_npmi)

In [9]:
model_trained_representation.visualize_topics()

In [20]:
(model_trained_representation.get_topic_info())

,Topic,Count,Name,Representation,Representative_Docs
0,-1,82942,-1_life_mom_love_wedding,"[life, mom, love, wedding, house, donald, day,...",[John Legend Says It Would Take A Gun To His H...
1,0,18986,0_fashion_dress_style_makeup,"[fashion, dress, style, makeup, miley, beauty,...",[Lena Dunham's McQ Dress On 'Girls' Shows Hann...
2,1,15829,1_trump_colbert_donald_trumps,"[trump, colbert, donald, trumps, bernie, presi...",[Seth Meyers Calls Out Donald Trump Jr. As The...
3,2,7611,2_foods_healthy_food_diet,"[foods, healthy, food, diet, nutrition, eating...","[Fast Food the Healthy Way: A Guide, Healthy F..."
4,3,6092,3_meditation_meditations_meditate_mindfulness,"[meditation, meditations, meditate, mindfulnes...","[Meditation Tips for the Day 1, How We Can Hel..."
5,4,4945,4_police_cops_officers_officer,"[police, cops, officers, officer, cop, violenc...",[Florida Cops On What Ferguson Can Learn From ...
6,5,4707,5_isis_syria_syrias_yemen,"[isis, syria, syrias, yemen, syrian, iran, isr...",[Israel Has Lost the War Against Hamas in Gaza...
7,6,4659,6_parenting_mothers_tweets_twitter,"[parenting, mothers, tweets, twitter, parental...",[Best Parenting Tweets: What Moms And Dads Sai...
8,7,4527,7_trumpcare_obamacare_health_medicaid,"[trumpcare, obamacare, health, medicaid, medic...",[Republicans Could Actually Pass This Health C...
9,8,3858,8_destinations_vacation_travelers_travel,"[destinations, vacation, travelers, travel, tr...","[Best New Travel Gadgets for 2014 (PHOTOS), 7 ..."
